In [129]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import pandas as pd
from scipy import fftpack
from numpy.fft import *
from numpy.random import seed
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from transforms3d.axangles import axangle2mat
import warnings
warnings.filterwarnings(action='ignore')

In [67]:
1875000/600

3125.0

In [40]:
def jerk_signal(signal, dt=0.02): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])
    
    
def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal


def feature_engineering(data_):
    data = data_.copy()
    data['acc_Energy']=(data['acc_x']**2+data['acc_y']**2+data['acc_z']**2)**(1/3)

    data['gy_Energy']=(data['gy_x']**2+data['gy_y']**2+data['gy_z']**2)**(1/3)

    data['gy_acc_Energy']=((data['gy_x']-data['acc_x'])**2+(data['gy_y']-data['acc_y'])**2+(data['gy_z']-data['acc_z'])**2)**(1/3)

    data_dt=[]
    for i in tqdm(data['id'].unique()):
        temp=data.loc[data['id']==i]
        for v in data.columns[2:]:
            values=jerk_signal(temp[v].values)
            values=np.insert(values,0,0)
            temp.loc[:,v+'_dt']=values
        data_dt.append(temp)
    data = pd.concat(data_dt)
    
    fft=[]
    for i in tqdm(range(len(data.shape[0]))):
        temp=data.loc[data['id']==i]
        for i in data.columns[2:8]:
            temp[i]=fourier_transform_one_signal(temp[i].values)
        fft.append(temp)
    data=pd.concat(fft)
    
    return data

In [41]:
def scaling(data_, scaler=None):
    data = data_
    col = data.columns
    
    if scaler:
        data.iloc[:,2:]= scaler.transform(data.iloc[:,2:])
        data = pd.DataFrame(data = data,columns =col)
        
        return data
    
    else:
        scaler = StandardScaler()
    
        data.iloc[:,2:]= scaler.fit_transform(data.iloc[:,2:])
        data = pd.DataFrame(data = data,columns =col)
        
        return data, scaler

In [102]:
def rolling(data_):
    data = data_.copy()
    sampling = np.random.choice(data.shape[0], int(data.shape[0] * 2 / 3))
    for j in sampling:
        data[j] = np.roll(data[j], np.random.choice(data.shape[1]), axis=0)
    return data


def rotation(data_):
    data = data_.copy()
    axis = np.random.uniform(low=-1, high=1, size=data.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(data, axangle2mat(axis, angle))


def permutation(data_, nPerm=4, mSL=10):
    data = data_.copy()
    data_new = np.zeros(data.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm + 1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(mSL, data.shape[0] - mSL, nPerm - 1))
        segs[-1] = data.shape[0]
        if np.min(segs[1:] - segs[0:-1]) > mSL:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        data_temp = data[segs[idx[ii]]:segs[idx[ii] + 1], :]
        data_new[pp:pp + len(data_temp), :] = data_temp
        pp += len(data_temp)
    return data_new


# 2:5
# 5:
def augmentation(data_):
    data = data_.copy()
    ro_data = data_.copy()
    per_data = data_.copy()
    
    # rotation
    print('rotation...')
    for j in tqdm(range(data.shape[0]//600)):
        # acc
        ro_data.iloc[j*600:(j+1)*600, 2:5] = rotation(np.array(data.iloc[j*600:(j+1)*600, 2:5]))
        # gy
        ro_data.iloc[j*600:(j+1)*600, 5:] = rotation(np.array(data.iloc[j*600:(j+1)*600, 5:]))
    
    ro_feature = feature_engineering(ro_data)
    ro_sc, _ = scaling(ro_feature)
    ro_series = make_series(ro_feature)
    ro_series = rolling(ro_series)
    
    # permutation
    print('permutation...')
    for j in tqdm(range(data.shape[0]//600)):
        # acc
        per_data.iloc[j*600:(j+1)*600, 2:5] = permutation(np.array(data.iloc[j*600:(j+1)*600, 2:5]))
        # gy
        per_data.iloc[j*600:(j+1)*600, 5:] = permutation(np.array(data.iloc[j*600:(j+1)*600, 5:]))

    per_feature = feature_engineering(per_data)
    per_sc, _ = scaling(per_feature)
    per_series = make_series(per_feature)
    per_series = rolling(per_series)
    
    origin_feature = feature_engineering(data)
    origin_sc, _ = scaling(origin_feature)
    origin_series = make_series(origin_feature)
    
    final = np.concatenate([origin_series, ro_series, per_series], axis=0)
    
    return final

In [103]:
def make_series(data):
    ids = data['id'].unique()
    id_data = data.groupby('id')
    series_data = []

    for i in ids:
        df = id_data.get_group(i).sort_values(['time'])
        df = df.drop(['id', 'time'], axis=1)
        series_data.append(df.to_numpy())

    series_data = np.array(series_data)
    return series_data

In [146]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [110]:
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [111]:
# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [112]:
def make_callbacks():
    # checkpoint path
    # 중간중간 모델의 weight를 저장할 경로 설정
    ckpt_name = 'aug_ckpt.hdf5'
    checkpoint_dir_path = os.path.join('checkpoint')
    checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)
    
    # callback 함수 목록
    callbacks_list = [
        # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
        tf.keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_weights_only=True,
            save_best_only=True
        ),
        # 8번 동안 val_loss의 향상이 없으면 훈련 종료
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            verbose=1, 
            patience=8
        ),
        tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
    ]
    
    return callbacks_list, checkpoint_path

In [114]:
from sklearn.model_selection import StratifiedKFold

In [147]:
labels = train_label['label'].to_numpy()

In [148]:
train_series = make_series(train)

In [158]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []

# id는 index다
for i, (train_id, val) in enumerate(split.split(train_series, labels)):
    callbacks_list, checkpoint_path = make_callbacks()
    print('Fold', i)
    print('#'*20)
    
    aug_data = augmentation(train[train['id'].isin(train_id)])
    train_labels = np.concatenate([labels[train_id], labels[train_id], labels[train_id]], axis=0)
    
    train_dataset = make_train(aug_data, train_labels)
    val_dataset = make_val(train_seires[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################
rotation...


100%|██████████████████████████████████████| 2812/2812 [00:05<00:00, 548.32it/s]


permutation...


100%|██████████████████████████████████████| 2812/2812 [00:05<00:00, 514.93it/s]


Epoch 1/100
132/132 [==============================] - 4s 30ms/step - loss: 2.2193 - accuracy: 0.5215 - val_loss: 1.8791 - val_accuracy: 0.5558
Epoch 2/100
132/132 [==============================] - 4s 28ms/step - loss: 1.5050 - accuracy: 0.6147 - val_loss: 1.4155 - val_accuracy: 0.6170
Epoch 3/100
132/132 [==============================] - 4s 28ms/step - loss: 1.2036 - accuracy: 0.6738 - val_loss: 1.1047 - val_accuracy: 0.6871
Epoch 4/100
132/132 [==============================] - 4s 28ms/step - loss: 1.0023 - accuracy: 0.7214 - val_loss: 0.9203 - val_accuracy: 0.7376
Epoch 5/100
132/132 [==============================] - 4s 28ms/step - loss: 0.8638 - accuracy: 0.7518 - val_loss: 0.7960 - val_accuracy: 0.7782
Epoch 6/100
132/132 [==============================] - 4s 29ms/step - loss: 0.7638 - accuracy: 0.7779 - val_loss: 0.7298 - val_accuracy: 0.7936
Epoch 7/100
132/132 [==============================] - 4s 29ms/step - loss: 0.6890 - accuracy: 0.7977 - val_loss: 0.6637 - val_accuracy:

...

Epoch 1/100
132/132 [==============================] - 4s 30ms/step - loss: 2.2082 - accuracy: 0.5244 - val_loss: 1.8339 - val_accuracy: 0.5573
Epoch 2/100
132/132 [==============================] - 4s 28ms/step - loss: 1.5034 - accuracy: 0.6181 - val_loss: 1.3924 - val_accuracy: 0.6255
Epoch 3/100
132/132 [==============================] - 4s 28ms/step - loss: 1.2104 - accuracy: 0.6713 - val_loss: 1.1067 - val_accuracy: 0.6913
Epoch 4/100
132/132 [==============================] - 4s 27ms/step - loss: 1.0050 - accuracy: 0.7164 - val_loss: 0.9198 - val_accuracy: 0.7374
Epoch 5/100
132/132 [==============================] - 4s 28ms/step - loss: 0.8668 - accuracy: 0.7509 - val_loss: 0.7925 - val_accuracy: 0.7726
Epoch 6/100
132/132 [==============================] - 4s 27ms/step - loss: 0.7660 - accuracy: 0.7790 - val_loss: 0.7072 - val_accuracy: 0.7971
Epoch 7/100
132/132 [==============================] - 4s 28ms/step - loss: 0.6915 - accuracy: 0.7996 - val_loss: 0.6482 - val_accuracy:

100%|██████████████████████████████████████| 2813/2813 [00:04<00:00, 589.14it/s]


permutation...


100%|██████████████████████████████████████| 2813/2813 [00:04<00:00, 597.39it/s]


Epoch 1/100
132/132 [==============================] - 4s 28ms/step - loss: 2.2157 - accuracy: 0.5215 - val_loss: 1.8160 - val_accuracy: 0.5571
Epoch 2/100
132/132 [==============================] - 3s 26ms/step - loss: 1.5062 - accuracy: 0.6139 - val_loss: 1.4217 - val_accuracy: 0.6164
Epoch 3/100
132/132 [==============================] - 4s 27ms/step - loss: 1.2114 - accuracy: 0.6675 - val_loss: 1.1286 - val_accuracy: 0.6827
Epoch 4/100
132/132 [==============================] - 4s 28ms/step - loss: 1.0115 - accuracy: 0.7170 - val_loss: 0.9377 - val_accuracy: 0.7303
Epoch 5/100
132/132 [==============================] - 4s 28ms/step - loss: 0.8745 - accuracy: 0.7497 - val_loss: 0.8123 - val_accuracy: 0.7754
Epoch 6/100
132/132 [==============================] - 4s 28ms/step - loss: 0.7741 - accuracy: 0.7802 - val_loss: 0.7511 - val_accuracy: 0.7990
Epoch 7/100
132/132 [==============================] - 4s 27ms/step - loss: 0.6998 - accuracy: 0.7989 - val_loss: 0.6921 - val_accuracy:

In [160]:
accs

[0.9723802804946899,
 0.9716690182685852,
 0.9665718078613281,
 0.9715505242347717,
 0.9684684872627258,
 0.9712051153182983,
 0.9674131870269775,
 0.970138669013977,
 0.9665837287902832,
 0.9627918004989624]

In [164]:
test_series = make_series(feature_engineering(test))

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 918.08it/s]


In [165]:
pred_list = []
for model in models:
    pred = model.predict(test_series)
    pred_list.append(pred)
pred = np.mean(pred_list, axis=0)

In [182]:
pred.argmax(axis=1)

array([50,  8,  8, 50,  8,  8,  8,  8, 15, 50, 50, 50, 50,  8,  8, 50,  8,
        8, 15, 50, 50, 50, 50, 50,  8,  8, 50,  8,  8, 50,  8, 50, 50,  8,
        8,  8, 50, 50,  8,  8,  8,  8,  8,  8,  8, 50, 50,  8, 50,  8, 50,
        8,  8, 50, 50,  8,  8, 50,  8,  8,  8,  8,  8, 50,  8, 50, 50, 50,
       50, 50,  8,  8,  8,  8,  8, 50, 50, 50, 50,  8,  8, 50,  8,  8, 50,
        8, 50, 50,  8, 50, 50,  8,  8,  8, 50,  8,  8, 50,  8,  8, 50, 50,
        8,  8,  8, 50, 50,  8,  8, 50, 50, 50,  8,  8,  8, 50,  8,  8,  8,
       50,  8,  8,  8, 50, 50,  8, 50, 42,  8, 15,  8, 50, 50,  8, 50,  8,
        8, 50, 50, 50,  8, 50, 50, 50, 15,  8,  8,  8, 50,  8,  8,  8,  8,
        8,  8,  8, 50,  8,  8, 15, 50,  8, 50,  8,  8,  8, 50,  8, 50,  8,
        8,  8,  8, 50,  8,  8,  8,  8, 50,  8,  8,  8, 50, 50, 50, 50,  8,
        8, 50, 50, 50, 50, 50,  8,  8,  8,  8, 50, 50,  8,  8,  8, 50,  8,
        8,  8,  8,  8, 50,  8,  8, 50,  8, 50, 50,  8,  8,  8,  8, 50, 50,
        8, 50, 50, 50,  8

In [183]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/aug.csv')     # 저장 경로
sub.to_csv(save_path, index=False)